In [12]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [13]:
from rake_nltk import Rake
import pandas as pd
import requests
from IPython.display import Image, display
from timeit import default_timer as timer
import pprint
import sys
from timeit import default_timer as timer
import gensim
import string

In [14]:
# Utilities
def print_song_keywords(Song_Name):
    rake = Rake()
    rake.extract_keywords_from_text(get_song(Song_Name))
    for i in range(0,len(rake.get_ranked_phrases_with_scores())):
        print(str(rake.get_ranked_phrases_with_scores()[i]))

def extract_song_keywords(Song_Name):
    rake = Rake()
    return rake.extract_keywords_from_text(get_song(Song_Name))

def get_song(title):
    return Songs.loc[title]["text"]

def print_song(title):
    print(get_song(title))
    
def createDict(keywords,N):
    Dict = {}
    r = requests.get('https://pixabay.com/api/?key=6713313-c85a6d8e3f6fe1a13b85f7d78&q='+"+".join(keywords)+'&image_type=photo&page=1&per_page='+str(N))
    for img in r.json()['hits']:
        Dict[img['webformatURL']] = img["tags"].split(",")
    return Dict

def displayDict(Dict,limit):
    i = 1
    urls = Dict.keys()
    
    for url in urls:
        print(url)
        print(Dict[url])
        display(Image(url,height=700,width=1000,embed=True))
        print("\n")
        if(i == limit): 
            break
        i += 1

In [15]:
breaker = "\n \n"
# Pre-processing
Songs = pd.read_csv("songdata.csv")
Songs.set_index("song",inplace=True)
Songs.drop(["artist","link"],inplace=True,axis=1)

In [16]:
rake = Rake()
rake.extract_keywords_from_text('Almost heaven, West Virginia,  Blue ridge mountain, Shenandoah river,  Life is old there, older than the trees,  Younger than the mountains, growing like a breeze ')
#rake.get_ranked_phrases_with_scores()

In [17]:
# Song Name -> List of URLS

Max_number_of_search_results = 200

start    = timer()
keywords = ['radio']
Dict     = createDict(keywords, Max_number_of_search_results)

print("\nTime taken:           " +  str(timer() - start) + "\nSize of Table Element:         " + str(len(Dict.keys())))


Time taken:           0.8757313877557209
Size of Table Element:         200


In [18]:
start = timer()
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)
print(str(timer() - start))

1.361292721690134


In [19]:
nlp = model.wv

In [57]:
tags = list(Dict.values())[3]

tags

['microphones', ' radio', ' sound']

In [58]:
# vector of similiar words to each tag
for tag in tags:
    # just a way to convert keywords into a form that can be submitted to a query
    filteredform = "".join((char for char in tag.lower().strip() if char not in string.punctuation))
    print(tag + "-> " + str(model.wv.most_similar(filteredform,topn=10)))
    print("\n")


microphones-> [('microphone', 0.7724970579147339), ('mics', 0.6849836707115173), ('headsets', 0.5621909499168396), ('earpiece', 0.559231698513031), ('headphones', 0.558610200881958), ('tape_recorder', 0.5561455488204956), ('cameras', 0.5552937984466553), ('camera', 0.5188095569610596), ('earphones', 0.5135119557380676), ('headset', 0.5106761455535889)]


 radio-> [('radio_stations', 0.7330180406570435), ('Radio', 0.7293578386306763), ('FM_radio', 0.6507300138473511), ('radio_broadcasts', 0.6375086307525635), ('FM', 0.629950761795044), ('broadcasting', 0.6133295297622681), ('television', 0.6114971041679382), ('broadcast', 0.6014235019683838), ('broadcasts', 0.5954024791717529), ('broadcaster', 0.5898584127426147)]


 sound-> [('sounds', 0.7409016489982605), ('sounding', 0.6120067834854126), ('sounded', 0.5613648295402527), ('noises', 0.5532740950584412), ('noise', 0.5079557299613953), ('Sounds', 0.5079108476638794), ('muffled', 0.5037038326263428), ('acoustics', 0.5034294128417969), ('S

In [29]:
s = "^^^^^^^^^^^Nigga!!!!!"

"".join((char for char in s.lower().strip() if char not in string.punctuation))

'nigga'

In [53]:
for i in range(0,len(Songs)):
    paras = Songs.iloc[i]["text"].split("\n \n")
    for para in paras:
        rake = Rake()
        kyws = rake.extract_keywords_from_text(para)
        
        for kyw in kyws:
            
            
        

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n"

In [81]:
tags

['microphones', ' radio', ' sound']

In [95]:
def get_mean_of_words(tags):
    mean = []
    for i in range(300):
         mean.append(0)
    size = len(tags)
    for tag in tags:
        mean += nlp.word_vec(tag.lower().replace(" ", "")) 
    
    mean = mean/size
    return mean

In [103]:
B = get_mean_of_words(tags)

In [113]:
A = (nlp.word_vec('microphones'.lower().replace(" ", "")) + nlp.word_vec('radio'.lower().replace(" ", "")) + nlp.word_vec('sound'.lower().replace(" ", "")))/3.00

In [114]:

print("A: " + str(A[10]) + " B: " + str(B[10]) + "\n")

A: -0.106283 B: -0.106282552083

